In [62]:
import pandas as pd
import numpy as np
import nltk
import string
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
colnames=['TweetId', 'Entity', 'Output', 'Tweet']

data = pd.read_csv('../input/twitter-entity-sentiment-analysis/twitter_validation.csv', names=colnames, header=None)

In [18]:
main = []

re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
stop_word  = set(stopwords.words('english'))
lem = WordNetLemmatizer()
p_stem = PorterStemmer()

for i in tqdm(range(len(data['Tweet']))):
    tokens = word_tokenize(str(data['Tweet'][i]))
    tokens = [w.lower() for w in tokens]
    tokens = [re_puncs.sub('', w) for w in tokens]
    tokens = [i for i in tokens if i.isalpha()]
    tokens = [w for w in tokens if w not in stop_word]
    tokens = [lem.lemmatize(w) for w in tokens]
    tokens = [p_stem.stem(w) for w in tokens]
    r = ' '.join(tokens)
    main.append(r)

100%|██████████| 1000/1000 [00:00<00:00, 1190.57it/s]


In [ ]:
data.head()

In [19]:
data['Preprocess_Tweet'] = main
data = data.drop('Tweet', axis = 1)

In [20]:
data.head()

,TweetId,Entity,Output,Preprocess_Tweet
0,3364,Facebook,Irrelevant,mention facebook struggl motiv go run day tran...
1,352,Amazon,Neutral,bbc news amazon bo jeff bezo reject claim comp...
2,8312,Microsoft,Negative,microsoft pay word function poorli samsungu ch...
3,4371,CS-GO,Negative,csgo matchmak full closet hack truli aw game
4,4433,Google,Neutral,presid slap american face realli commit unlaw ...


In [21]:
cnt = CountVectorizer(analyzer="word")
X = cnt.fit_transform(main).toarray()

In [22]:
data['Output'].unique()

array(['Irrelevant', 'Neutral', 'Negative', 'Positive'], dtype=object)

In [23]:
data['Output'].isnull().sum()

0

In [24]:
enc = OneHotEncoder(handle_unknown='ignore')
ip = np.array(data['Output'])
ip = ip.reshape(-1, 1)
y = enc.fit_transform(ip).toarray()

In [25]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [26]:
y

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [49]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [50]:
y_train

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [51]:
X_train.shape

(800, 3786)